# Agentic AI to fetch relevant Sikka APIs Using CREWAI

## 1. Import all the necessary libraries

In [1]:
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import os
from crewai import Agent, Task
from crewai.tools import tool
import json
from langchain_community.chat_models import ChatOllama

## 2. Load Sikka API CSV (containing API endpoints and its relevant information)

In [2]:
api_csv_path = "/Users/sudarshanc/Downloads/Sikka_APIs - Sikka_APIs.csv"
df = pd.read_csv(api_csv_path)

In [3]:
df.head()

,API Name,Description,API Endpoints,Document Link
0,appointments,Returns appointments data from practice,https://api.sikkasoft.com/v4/appointments,https://apidocs.sikkasoft.com/#cc4375ec-0b6a-4...
1,appointments_available_slots,Returns available appointments slots from prac...,https://api.sikkasoft.com/v4/appointments_avai...,https://apidocs.sikkasoft.com/#6b18ec4a-cfec-4...
2,accounts_receivables,Returns account receivables details from practice,https://api.sikkasoft.com/v4/accounts_receivables,https://apidocs.sikkasoft.com/#fcb005b7-b9ce-4...
3,accounts_receivables_by_patients,Returns account receivables details by patient...,https://api.sikkasoft.com/v4/accounts_receivab...,https://apidocs.sikkasoft.com/#3ae94fe1-4b24-4...
4,patients,Returns list of patients in practice,https://api.sikkasoft.com/v4/patients,https://apidocs.sikkasoft.com/#67aa35de-a4d4-4...


## 3. Create Document and Vector Store from the API csv

In [4]:
required_columns = {"API Name", "API Endpoints", "Description"} # making sure the data contains all these three columns 
if not required_columns.issubset(df.columns):
    raise KeyError(f"Missing required columns: {required_columns - set(df.columns)}")

In [5]:
faiss_index_path = "/Users/sudarshanc/faiss_index" # directory to store faiss index file/ vector embeddings 

embedding_model = OllamaEmbeddings(model="mistral") # Using Ollama(mistral LLM) to generate vector embeddings

docs = [] #list to store the data from the csv file
for _, row in df.iterrows():
    metadata = {
        "API_Name": row["API Name"],
        "API_Endpoint": row["API Endpoints"],
        "Description": row["Description"],
        "Documentation": row.get("Document Link", "N/A")
    }
    # creating a meaningful seentence to store data and metadata into docs list in terms of dictionary 
    document_text = f"Function: {row['Description']}. Use `{row['API Endpoints']}` to access it."
    
    docs.append(Document(page_content=document_text, metadata=metadata))

# Data Preprocessing to generate embeddings
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) # each chunk with 500 length and overlap of 50
split_docs = text_splitter.split_documents(docs) # splits the complete document based on hte text_splitter parameters

vectorstore = FAISS.from_documents(split_docs, embedding_model) # Generate the vector embeddings using Ollama Embeddings

vectorstore.save_local(faiss_index_path) # store the Faiss vector database in local directory(optional)

print("✅ FAISS Vector Store Created Successfully using Ollama!")

/var/folders/4k/zn3p378n3lz57m43gw71xxwm0000gn/T/ipykernel_28214/1489782109.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model="mistral") # Using Ollama(mistral LLM) to generate vector embeddings


✅ FAISS Vector Store Created Successfully using Ollama!


In [6]:
# Find the faiss vector embeddings from the local and load it for further computation
if os.path.exists(faiss_index_path):
    vectorstore = FAISS.load_local(faiss_index_path, embedding_model, allow_dangerous_deserialization=True)
    print("FAISS Vector Store Loaded Successfully!")
else:
    raise FileNotFoundError("FAISS Index Not Found! Make sure it is stored correctly.")

FAISS Vector Store Loaded Successfully!


## 4. Build Custom tool (Retriever Tool)

In [7]:
# Build a custom tool for retrieving relevant data from the vector database using crewai's tool decorator 
@tool("Retriever tool")
def retrieve_api_from_vectorstore(query: str):
    """Search the FAISS vector store for the most relevant API endpoint."""
    docs = vectorstore.similarity_search(query, k=1)  # Get the top API match
    if docs:
        return docs[0].metadata
    return "No relevant API found in FAISS vector store."

## 5. Define Retriever Agent and its Task

In [8]:
retriever_agent = Agent(
    role="Retriever", # Retriever role
    goal="Retrieve the most relevant Sikka API endpoint from FAISS based on user queries.", # goal of the agent
    backstory="""An AI-powered API documentation expert with deep knowledge of Sikka API services. 
    You are capable of using FAISS vector store to fetch relevant API details.""", 
    # explicitly indicating the agent to use Faiss database to fetch data
    description="Uses FAISS to retrieve structured API details, including endpoint, HTTP method, and description.", 
    llm="ollama/mistral",
    tools=[retrieve_api_from_vectorstore], # Providing our custom tool for the agent to perform the task 
    allow_delegation=False # This make sures that there is no external influence on the output i.e., influence of LLMs... 
)

In [9]:
retrieval_task = Task(
    description="Retrieve the best API endpoint for the given user request by searching FAISS vector store.",
    agent=retriever_agent, # Task to be used by which agent
    expected_output="A dictionary with API Name, Endpoint, HTTP Method, and Description." # output expectations from this task
)

## 6. Define Code Generator Agent and its Task
### To define this agent, we are providing a sample backend and frontend code templates in order to test the API endpoints wheather its accessible and what input or outputs does the endpoint is expecting.

In [10]:
# This tool generates the backend and frontend code depending on the user request and the retriever agents output
@tool("Dynamic Code Writer")
def generate_code(api_details: dict, user_request: str) -> str:
    """
    Uses LLM to generate full backend and frontend code purely from user request and retrieved API details.
    """
    # Prompt template
    prompt = f"""
You are a full-stack developer.

Your task is to write backend and frontend code to fulfill the following user request:
"{user_request}"

Here are the API details retrieved from the vector store:
{json.dumps(api_details, indent=2)}

1. First understand the user request.
2. Then decide what API call(s) are needed.
3. Generate a backend (Flask or FastAPI preferred) that will handle this request and make the API call.
4. Generate a frontend (React.js) that allows a user to interact with the backend.
5. Explain briefly how the code works.

Output only the code in two sections:
### Backend Code:
<code_here>

### Frontend Code:
<code_here>
"""

    llm = ChatOllama(model="mistral") # Define LLM
    response = llm.invoke(prompt) # invoke LLM wiht prompt template
    return response.content

In [11]:
code_generator_agent = Agent(
    role="Code Generator",
    goal="Generate dynamic, executable backend and frontend code tailored to the Sikka API.",
    backstory="An expert full-stack developer capable of analyzing API specifications and generating functional Flask and React.js code from scratch.",
    description="Generates fully functional backend and frontend code dynamically, based on API specifications and user queries.",
    llm="ollama/mistral",
    tools=[generate_code], # utilise code generation function
    allow_delegation=False
)

In [12]:
code_generation_task = Task(
    description="Dynamically generate full-stack (backend + frontend) code for the selected Sikka API based on user request.",
    agent=code_generator_agent,
    expected_output="Complete backend (Flask) and frontend (React.js) code dynamically generated based on API details."
)

## 7. Provide user query and invoke the agents to perform the tasks

In [13]:
user_query = "Display KPIs like total payments, outstanding balances, and last 30-day collections."
retrieved_api_info = retriever_agent.tools[0].run(user_query)  
print("🔍 API Details Retrieved:\n", retrieved_api_info)

Using Tool: Retriever tool
🔍 API Details Retrieved:
 {'API_Name': 'kpis', 'API_Endpoint': 'https://api.sikkasoft.com/v4/kpis', 'Description': 'A key performance indicator (KPI) is a business metric used to evaluate factors that are crucial to the success of an organization.KPIs give you unique views of practice data that lead to action items to drive increased revenue and save valuable practice time.', 'Documentation': 'https://apidocs.sikkasoft.com/#033c595a-15bb-4f52-91b2-cebff8c7cfc3'}


In [15]:
generated_code = code_generator_agent.tools[0].run(retrieved_api_info, user_query)
print("💻 Generated Full-Stack Code:\n", generated_code)

Using Tool: Dynamic Code Writer


/var/folders/4k/zn3p378n3lz57m43gw71xxwm0000gn/T/ipykernel_28214/3054442900.py:31: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="mistral") # Define LLM


💻 Generated Full-Stack Code:
  ### Backend Code (FastAPI):

```python
from fastapi import FastAPI, HTTPException
import requests

app = FastAPI()

@app.get("/kpis")
async def get_kpis():
    url = "https://api.sikkasoft.com/v4/kpis"
    headers = {"Authorization": "Bearer YOUR_API_KEY"}  # Replace with your API key
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise HTTPException(status_code=response.status_code, detail="Failed to fetch KPIs")

    data = response.json()
    total_payments = data["total_payments"]
    outstanding_balances = data["outstanding_balances"]
    collections_30days = data["collections_30days"]

    return {
        "total_payments": total_payments,
        "outstanding_balances": outstanding_balances,
        "collections_30days": collections_30days
    }
```

### Frontend Code (React.js):

```jsx
import React, { useState, useEffect } from 'react';
import axios from 'axios';

function KPIs() {
    const [kpis, s